ssh spsither@prodigy.bdrc.io

/home/spsither/staging/export_all.sh

tar -cJvf staging.tar staging/*.jsonl

In [ ]:
! scp spsither@prodigy.bdrc.io:/home/spsither/staging.tar data/

In [ ]:
! tar -xf data/staging.tar -C data/

In [ ]:
import pandas as pd
def read_jsonl(filename):
    df = pd.read_json(filename,lines=True)
    df = df.reset_index(drop=True)
    return df

In [ ]:
import os

jsonl_directory = "data/staging/"

files = os.listdir(jsonl_directory)

pattern = "jsonl"
files_matching_pattern = [file for file in files if file.endswith(pattern)]

In [ ]:
#all reviewed files 
reviewed_jsonl_files = []
for file in files_matching_pattern:
    if 'review' in file and not 'stt_second_review' in file:
        reviewed_jsonl_files.append(file)

In [ ]:
#all unreviewed files
transcribed_jsonl_files = []
for file in files_matching_pattern:
    if 'review' not in file and not 'stt_second_review' in file:
        transcribed_jsonl_files.append(file)

In [ ]:
! pip install tqdm
! pip install ipywidgets

In [ ]:
#concatenate all the transcribed dataframes
temp = pd.DataFrame([])
from tqdm.auto import tqdm
for file in tqdm(transcribed_jsonl_files):
    df = read_jsonl(f"{jsonl_directory}/{file}")
    temp = pd.concat([temp,df],axis=0)
    trabscribed_df = temp.reset_index(drop=True)

In [ ]:
#concatenate all the reviewed dataframes
from tqdm.auto import tqdm
temp = pd.DataFrame([])
for file in tqdm(reviewed_jsonl_files):
    df = read_jsonl(f"{jsonl_directory}/{file}")
    temp = pd.concat([temp,df],axis=0)
    reviewed_df = temp.reset_index(drop=True)

In [ ]:
finale_df = read_jsonl(f"{jsonl_directory}/stt_second_review.jsonl")

In [ ]:
trabscribed_df['id'].fillna(trabscribed_df['text'], inplace=True)
reviewed_df['id'].fillna(reviewed_df['text'], inplace=True)
finale_df['id'].fillna(finale_df['text'], inplace=True)

In [ ]:
audio_extensions = ['.mp3', '.wav', '.MP3', '.WAV']

In [ ]:
for audio_extension in audio_extensions:
    trabscribed_df['id'] = trabscribed_df['id'].str.replace(audio_extension, '')
    reviewed_df['id'] = reviewed_df['id'].str.replace(audio_extension, '')
    finale_df['id'] = finale_df['id'].str.replace(audio_extension, '')

In [ ]:
len(trabscribed_df),len(reviewed_df),len(finale_df)

In [ ]:
len(set(trabscribed_df['id'])), len(set(reviewed_df['id'])), len(set(finale_df['id']))

In [ ]:
100 - len(set(trabscribed_df['id'])) / len(trabscribed_df) * 100

In [ ]:
trabscribed_df.shape[0], reviewed_df.shape[0], finale_df.shape[0]

In [ ]:
trabscribed_df.drop_duplicates(subset='id', keep="first", inplace=True)
reviewed_df.drop_duplicates(subset='id', keep="first", inplace=True)
finale_df.drop_duplicates(subset='id', keep="first", inplace=True)

In [ ]:
trabscribed_df.shape[0], reviewed_df.shape[0], finale_df.shape[0]

In [ ]:
len(trabscribed_df),len(reviewed_df),len(finale_df)

In [ ]:
trabscribed_df = trabscribed_df[trabscribed_df['answer'] == 'accept']
reviewed_df = reviewed_df[reviewed_df['answer'] == 'accept']
finale_df = finale_df[finale_df['answer'] == 'accept']

In [ ]:
len(trabscribed_df),len(reviewed_df),len(finale_df)

In [ ]:
# leakage check
len(trabscribed_df) - len(finale_df), len(trabscribed_df) - len(reviewed_df)

In [ ]:
# left side intersection
intersection = reviewed_df.merge(finale_df, how='left', on='id')

In [ ]:
intersection['grade'] = 1

In [ ]:
intersection.loc[~intersection['transcript_x'].isna(), 'grade'] = 2

In [ ]:
intersection.loc[~intersection['transcript_y'].isna(), 'grade'] = 3

In [ ]:
intersection['transcript_y'].fillna(intersection['transcript_x'], inplace=True)


In [ ]:
intersection = intersection[~intersection['transcript_y'].isna()]

In [ ]:
intersection = intersection[['transcript_y', 'id', 'grade']]

In [ ]:
intersection['url'] = 'https://d38pmlk0v88drf.cloudfront.net/wav/' + intersection['id'] + '.wav'

In [ ]:
intersection.shape

In [ ]:
i = 11001
intersection.loc[i]['transcript_y'], intersection.loc[i,'url']

In [ ]:
intersection['dept'] = intersection['id'].str[:6]

In [ ]:
intersection.groupby('dept').size()

In [ ]:
intersection.rename(columns={"id": "file_name", "transcript_y": "uni", "url": "url", "dept": "dept", "grade": "grade"}, inplace=True)

In [ ]:
last_intersection = intersection[['file_name', 'uni', 'url', 'dept', 'grade']]
last_intersection.head()

In [ ]:
last_intersection.to_csv('02_prodigy.tsv', sep='\t', index=False)

In [ ]:
last_intersection[last_intersection['file_name'] == 'STT_AB00001_0029_150584_to_151748']